# CARGA DE DADOS DO BIRH
---
O BIRH é a base de dados integrada de RH da Petrobras.

Esta base de dados será consultada para extração de dados corporativos não sensíveis dos usuários


In [ ]:
import os
import os.path
import cx_Oracle
import sqlalchemy
import pandas as pd

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [ ]:
oracle_host       = os.environ['ORACLE_HOST']
oracle_port       = os.environ['ORACLE_PORT']
oracle_service    = os.environ['ORACLE_SERVICE']
oracle_user       = os.environ['ORACLE_USER']
oracle_password   = os.environ['ORACLE_PASSWORD']
oracle_home       = os.environ['ORACLE_HOME']

A biblioteca cx_Oracle requer a instação de um Oracle Client contendo bibliotecas C necessária para comunicação com o servidor.

A maneira mais fácil de atender esse requisito é por meio da instalação do Oracle Instant Client, que pode ser baixada e descompactada na pasta do próprio projeto.

Segue link abaixo:

https://www.oracle.com/br/database/technologies/instant-client/winx64-64-downloads.html

Após instalação, é necessário indicar na instalação da biblioteca onde a mesma encontra-se

In [ ]:
cx_Oracle.init_oracle_client(lib_dir=oracle_home)

A consulta abaixo retorna a lista de usuários com chave ativa, trazendo tanto funcionários próprios como concusrsados.

Nenhuma informação sensível sobre os mesmos está sendo retornada, nem mesmo o nome nome das pessoas

In [ ]:
SQL_DADOS_BIRH = """
    SELECT  a.fotr_cd_chave                                     AS chave_usuario
    ,       UPPER(a.tift_nm_tipo_forca_trabalho)                AS tipo_usuario
    ,       a.lota_cd_centro_custo                              AS centro_custo
    ,       NVL(REGEXP_SUBSTR(
                unor_sg_unidade_organizacional
            ,   '(^[^/]+)'
            ,   1     -- Start at the first character
            ,   1     -- Get the first match
            ,   NULL  -- Case sensitive
            ,   1     -- Return the first capture group
            ), 'n/a')                                           AS lotacao_topo
    ,       NVL(UPPER(a.unor_sg_unidade_organizacional), 'n/a') AS sigla_lotacao
    ,       NVL(UPPER(a.unor_nm_unidade_organizacional), 'n/a') AS nome_lotacao
    ,       NVL(UPPER(a.carg_nm_cargo), 'n/a')                  AS cargo
    ,       NVL(UPPER(a.enfa_nm_enfase), 'n/a')                 AS enfase
    ,       NVL(UPPER(a.func_nm_funcao), 'n/a')                 AS funcao
    ,       UPPER(a.sind_nm_sindicato)                          AS sindicato
    ,       UPPER(a.arrh_nm_area_rh)                            AS area_rh
    ,       UPPER(a.ftre_nm_imovel_subarea_rh)                  AS imovel
    ,       UPPER(a.lone_nm_local_negocio)                      AS local_negocio
    ,       UPPER(a.grap_nm_grupo_ativ_prest_serv)              AS grupo_prestacao_servico
    ,       UPPER(a.retr_ds_regime_trab_permanente)             AS regime_trabalho
    ,       UPPER(a.emgp_nm_empresa_contrato)                   AS empresa_contrato
    ,       UPPER(a.prtl_in_tipo_localizacao)                   AS tipo_localizacao
    ,       a.pcse_in_status_cracha                             AS status_cracha
    ,       a.pcse_in_situacao_cracha                           AS situacao_cracha
    FROM    tcrp.MV_BIRH_FOTR_PRSE_VIGENTE a -- MVIEW do BIRH Força de Trabalho & Prestadores de Serviços Vigentes
    WHERE   a.emgp_cd_sap_empresa                   = '1000'
    AND     a.fotr_in_estado_chave                  = 'A'
    AND     a.siat_cd_situacao                      = 'A'
"""

Estabelecimento da conexão usando sqlalchemy. A função **pandas.read_sql** utiliza sqlalchemy para conversar com qualquer banco de dados (além do sqlite)

In [ ]:
oracle_dsn        = cx_Oracle.makedsn(oracle_host, oracle_port , oracle_service).replace('SID', 'SERVICE_NAME')
oracle_connstring = f'oracle://{oracle_user}:{oracle_password}@{oracle_dsn}'
engine            = sqlalchemy.create_engine(oracle_connstring)
conn              = engine.connect()

Extração do conjunto de dados

In [ ]:
birh_df = pd.read_sql(SQL_DADOS_BIRH, conn)
birh_df.info(verbose=True, show_counts=True)

Salvando o conjunto de dados como um arquivo parquet para poder posteriormente consultá-lo usando a biblioteca **DuckDB**

In [ ]:
birh_df.to_parquet(os.environ['BIRH_DATA'])

In [ ]:
print(f"arquivo do BIRH gerado -> {os.environ['BIRH_DATA']}")